# Converting MCNP Models into OpenMC Models

MCNP is an established Monte Carlo transport code used in many organizations.
MCNP models can be converted into OpenMC models using the `openmc_mcnp_adapter`, freely available on GitHub: https://github.com/openmc-dev/openmc_mcnp_adapter. 

In this tutorial, we will use this adapter to load in an existing MCNP model of a High Purity Germanium (HPGe) detector to perform a photon transport simulation. This MCNP model was originally developed by Z. Liu and A. Di Fulvio from the University of Illinois, Urbana-Champaign. Full details on the model can be found here: https://www.sciencedirect.com/science/article/pii/S0168900224005436?via%3Dihub

In [ ]:
import openmc
import numpy as np
import matplotlib.pyplot as plt

First, we need to have the `openmc_mcnp_adapter` installed (follow directions here: https://github.com/openmc-dev/openmc_mcnp_adapter). Then, we need to have an MCNP model; this is contained in the `mcnp.i` file, which we can view.

In [ ]:
!cat mcnp.i

When we run the converter, we will create a file named `mcnp.xml`. We can then load this from XML to access the model data structures from Python. This will create an OpenMC model.

```
mcnp_to_openmc mcnp.i -o mcnp.xml
```

There are some important limitations of the MCNP model converter. At present, this converter does *not* convert:

- Source definitions
- Tallies
- Graveyard boundary conditions
- Certain geometry structures (such as hexagonal lattices and certain macrobodies)

Therefore, we will import this model and make some modifications to re-add source definitions, tallies, and enclose that MCNP model inside a vacuum boundary surface (if a graveyard was used).
For a full list of unsupported features, see: https://github.com/openmc-dev/openmc_mcnp_adapter

In [ ]:
mcnp_model = openmc.Model.from_model_xml("mcnp.xml")

In [ ]:
type(mcnp_model)

For example, we can inspect the materials in the MCNP model.

In [ ]:
mcnp_materials = mcnp_model.materials
print(mcnp_materials)

This HPGe consists of material cells, surrounded by a vacuum.

In [ ]:
mcnp_model.plot(width=(50, 90), pixels=100000, basis='xz', origin = (0, 0, 140), color_by='material')

We can get the root universe of the MCNP model, in case we want to place that geometry inside something more extensive in our OpenMC model. Let's place the MCNP universe inside a sphere with a vacuum boundary on its exterior and then create a new model whose geometry is the MCNP model inside the sphere.

In [ ]:
mcnp_univ = mcnp_model.geometry.root_universe
sphere = openmc.Sphere(r=50, x0=0.0, y0=0.0, z0=140, boundary_type='vacuum')
sphere_cell = openmc.Cell(region=-sphere, fill=mcnp_univ)

In [ ]:
model = openmc.Model()
u = openmc.Universe(cells=[sphere_cell])
model.geometry = openmc.Geometry(u)

model.plot(width=(150, 150), pixels=100000, basis='xz', origin = (0, 0, 140), color_by='cell')

In [ ]:
model.settings = openmc.Settings()
model.settings.particles = 5000
model.settings.batches = 100
model.settings.run_mode = 'fixed source'

The source definition does not get converted from the MCNP model; let's add a point source of 661.7 keV photons, located at (0, 0, 100).

In [ ]:
source = openmc.IndependentSource(
    energy=openmc.stats.Discrete([661.7e3], [1]),
    space=openmc.stats.Point((0, 0, 100.0)),
    angle=openmc.stats.Isotropic(),
    particle="photon"
)

model.settings.source = source
model.settings.photon_transport = True

We could then also add any tallies we wish to have; we can add a `pulse-height` tally to record the photon energy deposited in the germanium cells. First, we need to find which cells contain germanium.

In [ ]:
type(mcnp_model.geometry.get_all_cells())

In [ ]:
ge_cell_ids = []

cells = mcnp_model.geometry.get_all_cells()
for id in cells:
    cell_fill = cells[id].fill

    if cell_fill is None:
        continue

    has_ge = False
    if type(cell_fill) is openmc.Material:
        nuclides = cell_fill.nuclides
        for n in nuclides:
            string = n.name
            without_numbers = ''.join([i for i in string if not i.isdigit()])
            if (without_numbers == 'Ge'):
                ge_cell_ids.append(id)

In [ ]:
ge_cell_ids = np.unique(ge_cell_ids)
print(ge_cell_ids)

cell_filter = openmc.CellFilter(ge_cell_ids)
energy_filter = openmc.EnergyFilter(np.linspace(0, 1e6, 1000))
tally = openmc.Tally()
tally.scores = ['pulse-height']
tally.filters = [cell_filter, energy_filter]

model.tallies = openmc.Tallies([tally])

In [ ]:
statepoint = model.run()

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    ph_tally = sp.get_tally(id=tally.id)

ph_tally.get_reshaped_data().shape
ph_tally = ph_tally.get_reshaped_data().squeeze()

In [ ]:
plt.plot(np.unique(energy_filter.bins)[:-1], ph_tally[0,:])
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Energy (eV)')
plt.ylabel('Pulse Height Tally (eV/src)')
plt.grid()
plt.show()